### Transform Sessions 

Goal: 

Extract the session data, transform it into a dataframe that can be compared to and used with other part of the app 

In [1]:
import pandas as  pd 
from xml.etree import ElementTree


from pathlib import Path
import os
import re


In [2]:
# config variables

ROOT_DIR = Path('~/Projects/Data/ccrsApp/backend')
DATA_DIR = ROOT_DIR/'data'

In [3]:
# Funcitons 




def is_cjk(char):
    char = ord(char)
    cjk_ranges = [
    (0x4E00,  0x62FF),
    (0x6300,  0x77FF),
    (0x7800,  0x8CFF),
    (0x8D00,  0x9FCC),
    (0x3400,  0x4DB5),
    (0x20000, 0x215FF),
    (0x21600, 0x230FF),
    (0x23100, 0x245FF),
    (0x24600, 0x260FF),
    (0x26100, 0x275FF),
    (0x27600, 0x290FF),
    (0x29100, 0x2A6DF),
    (0x2A700, 0x2B734),
    (0x2B740, 0x2B81D),
    (0x2B820, 0x2CEAF),
    (0x2CEB0, 0x2EBEF),
    (0x2F800, 0x2FA1F), ]

    
    
    for bottom, top in cjk_ranges:
        if char >= bottom and char <= top:
            return True
    return False

# Adds new words
def new_vocab_df_generator(elem):
    """_summary_
    
    Takes in an xml element, returns a dataframe of the chinese characters in each element
    along with the date
    
    Added
    
    Args:
        elem (_type_): _description_
        xml element, expected to be in the opml format from workflowy

    Returns:
        _type_: _description_
    """
    
    #vocab_df = pd.DataFrame({}, columns=['Characters', 'Date'])

    vocabFromCat = []
    catName = elem.attrib['text']
    for lessonNotes in elem.iter():
        # print(elementsDateAdded[i])
        
        for note in lessonNotes:
            #print(note)
            noteText = note.attrib['text']

            chineseCharacters = ''

            for char in noteText:
                if is_cjk(char):
                    chineseCharacters += char

            vocabFromCat.append(chineseCharacters)

        # Generates a column of same shape as vocab
        #dateColumn = [elementsDateAdded[i]] * len(vocabFromLesson)
        
        #lessonDictionary = dict(zipvocabFromLesson)
        catCol = [catName] * len(vocabFromCat)
        #catDictionary = dict(zip(vocabFromCat,catCol))
        tmp_df = pd.DataFrame({'Characters':vocabFromCat,
                               'Cat':catCol},dtype='object')
        print(tmp_df.shape)
        return tmp_df


In [4]:
fileName = '../data/tutor_session_notes.opml'

In [5]:
with open(fileName, 'rt') as f:
        tree = ElementTree.parse(f)

notesRoot = tree.getroot()

notesBody = notesRoot.find('body')

In [6]:
def find_child_element(child_elem_text,parent_elem):
    childElement = None

    for elem in parent_elem:
        #print(elem.attrib['text'] )

        if elem.attrib['text']  is not None and elem.attrib['text'].strip() == child_elem_text:
            childElement  = elem
            break
    
    if childElement is not None:
        return childElement
    else: 
        print('No Elem Found')
        return None


In [7]:
next_child =  find_child_element('<span class="colored bc-pink">Knowledge</span>',notesBody)

next_child = find_child_element('Active Subjects',next_child)
next_child = find_child_element('Chinese #huayu', next_child)
next_child =  find_child_element('好好学习，天天上上',next_child)
tutor_session_notes = find_child_element('Tutor Session Notes',next_child)

In [8]:

# Looks for element text which matches internal 
# workflowy pattern in opml 
timePattern = re.compile('time startYear')

# Creating list of elements
# Which contain lesson notes 

session_dict = {}


# Checks for elements which contain dates
# Which will contain sub elements with vocab
# Appends them to a list, along with the dates
# From before

for outline in tutor_session_notes.iter():

    text = outline.attrib['text']
    #Extracts the Date 
    # finds new nodes
    
    hasTime = re.search(timePattern,text)
    
    if hasTime:
        #Extracting datetime info
        time = text.split()
        year = time[1].split('\"')[1]
        month = time[2].split('\"')[1]
        day = time[3].split('\"')[1]
        dateAdded = year+' '+month +' '+day
        #print(dateAdded)
        
        session_dict[dateAdded] = outline

    else:
        continue 

#print(elementsByDate)


In [35]:
re.search('...','... ...')[0]

'...'

AttributeError: module 're' has no attribute 'find'

In [9]:
testSession = session_dict['2023 4 5']

In [49]:
re…search

SyntaxError: invalid character '…' (U+2026) (4117168034.py, line 1)

In [68]:
import jieba.posseg as pseg

def pos_tag(text):
    words = pseg.cut(text)
    tags = []
    for word, tag in words:
        tags.append((word, tag))
    return tags

def count_parts_of_speech(text):
    tag_counts = {}
    tags = pos_tag(text)
    for word, tag in tags:
        if tag in tag_counts:
            tag_counts[tag] += 1
        else:
            tag_counts[tag] = 1
    return len(tag_counts)


In [71]:
len(None)

TypeError: object of type 'NoneType' has no len()

In [99]:
len(session_dict)
import unicodedata

#is_example_grammar

lessonColumns = ['characters','full_text']
df_lesson = pd.DataFrame({},lessonColumns)

# To concat at the end
session_df_list = []

for session_date, session_notes in session_dict.items():
    chineseCharactersList = []
    lesson_notes_text_list = []
    # Extracts Chinese Charcters
    for noteIdx, lesson_note in enumerate(session_notes.iter()):
        # skips first note that contains date information
        if (noteIdx == 0):
            continue
        
        lesson_note.attrib['text'] = unicodedata.normalize('NFC', lesson_note.attrib['text'])

        chineseCharacters = ''
        for char in lesson_note.attrib['text']:
            if is_cjk(char):
                chineseCharacters += char
        
        # adding to a list for later, none to keep 
        if len(chineseCharacters) > 0: 
            chineseCharactersList.append(chineseCharacters)
        else:
            chineseCharactersList.append('')
        lesson_notes_text_list.append(lesson_note.attrib['text'])
    
    assert len(chineseCharactersList) == len(lesson_notes_text_list)

    # determine the type of note

    noteTypeList = []
    
    for noteIdx, noteText in enumerate(lesson_notes_text_list):

        note_chinese = chineseCharactersList[noteIdx]

        if len(note_chinese) == 0:
            noteTypeList.append('English')
        
        elif len(note_chinese) < 5:
            noteTypeList.append('Vocab')
        
        elif len(re.findall('\.\.|…',noteText.replace(' ',''))) > 2 :
            noteTypeList.append('Grammar')
        
        elif len(note_chinese) > 4:
            unique_parts_of_speech =  count_parts_of_speech(note_chinese)
            if unique_parts_of_speech > 3:
                noteTypeList.append('Sentence')
            else:
                noteTypeList.append('Phrase')
        else:
            noteTypeList.append('?')
        
    assert len(noteTypeList) == len(chineseCharactersList)
    
    df_indv_session =  pd.DataFrame({'hanzi': chineseCharactersList,
                                    'full_text': lesson_notes_text_list,
                                    'note_type': noteTypeList,
                                    'date' : session_date })
    
    session_df_list.append(df_indv_session)

df_lesson_notes = pd.concat(session_df_list)
df_lesson_notes = df_lesson_notes[df_lesson_notes.full_text.str.len() > 0]



In [100]:
df_lesson_notes.reset_index(drop=True,inplace=True)

In [102]:
df_lesson_notes.to_csv('../data/lesson_notes_2023_04_12')

<table>
<thead>
<tr>
<th>标签</th>
<th>含义</th>
<th>标签</th>
<th>含义</th>
<th>标签</th>
<th>含义</th>
<th>标签</th>
<th>含义</th>
</tr>
</thead>
<tbody>
<tr>
<td>n</td>
<td>普通名词</td>
<td>f</td>
<td>方位名词</td>
<td>s</td>
<td>处所名词</td>
<td>t</td>
<td>时间</td>
</tr>
<tr>
<td>nr</td>
<td>人名</td>
<td>ns</td>
<td>地名</td>
<td>nt</td>
<td>机构名</td>
<td>nw</td>
<td>作品名</td>
</tr>
<tr>
<td>nz</td>
<td>其他专名</td>
<td>v</td>
<td>普通动词</td>
<td>vd</td>
<td>动副词</td>
<td>vn</td>
<td>名动词</td>
</tr>
<tr>
<td>a</td>
<td>形容词</td>
<td>ad</td>
<td>副形词</td>
<td>an</td>
<td>名形词</td>
<td>d</td>
<td>副词</td>
</tr>
<tr>
<td>m</td>
<td>数量词</td>
<td>q</td>
<td>量词</td>
<td>r</td>
<td>代词</td>
<td>p</td>
<td>介词</td>
</tr>
<tr>
<td>c</td>
<td>连词</td>
<td>u</td>
<td>助词</td>
<td>xc</td>
<td>其他虚词</td>
<td>w</td>
<td>标点符号</td>
</tr>
<tr>
<td>PER</td>
<td>人名</td>
<td>LOC</td>
<td>地名</td>
<td>ORG</td>
<td>机构名</td>
<td>TIME</td>
<td>时间</td>
</tr>
</tbody>
</table>

In [ ]:
# Adds new words
def new_vocab_df_generator(elem):
    """_summary_
    
    Takes in an xml element, returns a dataframe of the chinese characters in each element
    along with the date
    
    Added
    
    Args:
        elem (_type_): _description_
        xml element, expected to be in the opml format from workflowy

    Returns:
        _type_: _description_
    """
    
    vocab_df = pd.DataFrame({}, columns=['Characters', 'Date'])

    for i, lessonNotes in new_sessions.iter:

        vocabFromLesson = []

        # print(elementsDateAdded[i])
        for note in lessonNotes:
            noteText = note.attrib['text']

            chineseCharacters = ''

            for char in noteText:
                if is_cjk(char):
                    chineseCharacters += char

            vocabFromLesson.append(chineseCharacters)

        # Generates a column of same shape as vocab
        dateColumn = [elementsDateAdded[i]] * len(vocabFromLesson)
        lessonDictionary = {'Characters': vocabFromLesson,
                            'Date': dateColumn}
        # dict(zip(vocabFromLesson,dateColumn))

        df_dictionary = pd.DataFrame(lessonDictionary, columns=['Characters', 'Date'])

        return df_dictionary